In [25]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from bullinger import chronogram
from bullinger import cohort
from bullinger import intervals
from bullinger import video

intervals = importlib.reload(intervals)
video = importlib.reload(video)
chronogram = importlib.reload(chronogram)
cohort = importlib.reload(cohort)

In [26]:
folder = '/home/olivier/dev/data/laura_codage/'
c = cohort.Cohort(folder)
c.df

,index,actor,video_id,start,end,duration,tag,semester,name,group,context,num_supports
0,36,contexte,Francesco_(6-12)_13,1.026,1.036,0.010,support,2.0,Francesco,TD,veh,1.0
1,34,contexte,Francesco_(6-12)_13,1.036,2.626,1.590,support,2.0,Francesco,TD,"pip,veh",2.0
2,11,adulte,Francesco_(6-12)_13,2.250,2.626,0.376,parle,2.0,Francesco,TD,"pip,veh",2.0
3,24,bebe,Francesco_(6-12)_13,2.300,2.626,0.326,init,2.0,Francesco,TD,"pip,veh",2.0
4,13,adulte,Francesco_(6-12)_13,2.626,4.501,1.875,parle,2.0,Francesco,TD,pip,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4413,8,bebe,Carlo_(6-12)_10,62.217,64.697,2.480,init,2.0,Carlo,AD,bap,1.0
4414,9,bebe,Carlo_(6-12)_10,64.697,64.790,0.093,init,2.0,Carlo,AD,sans,0.0
4415,13,contexte,Carlo_(6-12)_10,64.697,64.790,0.093,support,2.0,Carlo,AD,sans,0.0
4416,4,contexte,Carlo_(0-6)_3,0.000,104.986,104.986,support,1.0,Carlo,AD,"bap,dop,tep",3.0


In [27]:
print(c.num_annotations, 'annotations in', len(c), 'videos' )

3320 annotations in 196 videos


In [41]:
c.context_df.groupby(['semester', 'group']).agg({'video_id': 'nunique', 'name': 'nunique', 'duration': np.sum})

video_id  name  duration
semester group                          
1.0      AD           52    10  5141.813
         TD           49    10  4394.067
2.0      AD           46    10  4657.678
         TD           47    10  3777.189

In [22]:
sum(v.duration for v in c)

17970.747000000003